In [96]:
import subprocess
import os
import argparse
import pandas as pd
import numpy as np
import requests
from sqlalchemy import create_engine
from dotenv import load_dotenv
import logging
from sqlalchemy import text
import duckdb
import datetime
from bs4 import BeautifulSoup
pd.set_option('display.max_columns', None)

load_dotenv()
DATABASE_URL = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
engine = create_engine(DATABASE_URL)

In [97]:
engine 
start_year=2024 
end_year=2024 
parquet_folder = 'data/raw/nflfastr/'

In [98]:
weekly_df = pd.read_sql("SELECT * FROM weekly_stats", engine)
players_df = pd.read_sql("SELECT * FROM players", engine)
teams_df = pd.read_sql('SELECT * FROM teams', engine)
games_df = pd.read_sql('SELECT * FROM games', engine)
depth_df = pd.read_sql('SELECT * FROM depth_chart', engine)
injuries_df = pd.read_sql('SELECT * FROM injuries', engine)
schedule_df = pd.read_sql("SELECT season, week, stadium, game_date FROM games", engine)
base_features_df = pd.read_sql("SELECT * FROM player_weekly_features", engine)
pre_features_df = pd.read_sql("SELECT * FROM player_weekly_features", engine)
features_df = pd.read_sql("SELECT * FROM features", engine)

In [115]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

season = 2023

load_dotenv()
engine = engine
url = "https://nfllines.com/offensive-line-rankings/"
r = requests.get(url, timeout=10)
r.raise_for_status()
soup = BeautifulSoup(r.text, "html.parser")
table = soup.find("table")
df = pd.read_html(str(table))[0]
df.columns = df.iloc[0]
df = df[1:]

df = df.rename(columns={
    'Team': 'abbreviation',
    'Pipeline Overall Win Rate (%)': 'ol_win_rate',  
    'Pass Blocking (%)': 'pass_block_rate',
    'Run Blocking (%)': 'run_block_rate'
})

df['season'] = season
# If there's a week column, include, otherwise infer week based on date or manual input
df['week'] = pd.NA
df['source'] = 'NFLLines'

teams_df = pd.read_sql("SELECT team_id, abbreviation FROM teams", engine)
df = df.merge(teams_df, how='left', on='abbreviation')
df = df.dropna(subset=['team_id'])

final = df[['season', 'week', 'team_id', 'ol_win_rate', 'pass_block_rate', 'run_block_rate', 'source']]

/var/folders/w3/190dcng90jb_w86_k9pqrdqh0000gn/T/ipykernel_61709/274730671.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


KeyError: "['ol_win_rate', 'pass_block_rate', 'run_block_rate'] not in index"

In [113]:
load_dotenv()
engine = engine
url = "https://nfllines.com/offensive-line-rankings/"
r = requests.get(url, timeout=10)
r.raise_for_status()
soup = BeautifulSoup(r.text, "html.parser")
table = soup.find("table")
df = pd.read_html(str(table))[0]
df.columns = df.iloc[0]
df = df[1:]

/var/folders/w3/190dcng90jb_w86_k9pqrdqh0000gn/T/ipykernel_61709/3479838054.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


In [114]:
df

,Team,wk3 Score,wk 3 Rank,wk4 Score,wk4 Rank,Chg Score,Chg Rank
1,Baltimore Ravens,12.10,1,12.47,1,0.36,0
2,Washington Commanders,3.71,12,8.01,2,4.30,10
3,Philadelphia Eagles,11.39,2,7.22,3,-4.17,-1
4,Kansas City Chiefs,6.35,7,6.81,4,0.46,3
5,Cincinnati Bengals,7.86,5,6.53,5,-1.33,0
6,Detroit Lions,4.71,8,6.09,6,1.38,2
7,Green Bay Packers,4.04,10,5.21,7,1.17,3
8,Buffalo Bills,10.09,3,4.25,8,-5.84,-5
9,San Francisco 49ers,4.23,9,4.05,9,-0.18,0
10,New Orleans Saints,8.89,4,4.02,10,-4.88,-6


In [124]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

def fetch_defensive_rankings(engine, season: int, unit: str):
    """
    unit: 'front-seven' or 'secondary'
    """
    url = f"https://www.sharpfootballanalysis.com/analysis/best-nfl-{unit}-rankings/"
    resp = requests.get(url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")
    all_tables = soup.find_all("table")
    
    # Pick the correct table (the first one with a 'Team' column)
    df = None
    for table in all_tables:
        tdf = pd.read_html(str(table))[0]
        if "Team" in tdf.columns and "Score" in tdf.columns:
            df = tdf
            break
    
    if df is None:
        raise ValueError("❌ Could not find the rankings table with 'Team' and 'Score' columns.")
    
    df = df.rename(columns={"Team": "team_name", "Score": f"{unit}_score"})
    df["season"] = season
    df["source"] = "SharpFootball"

    # Join to team table
    teams_df = pd.read_sql("SELECT team_id, full_name, abbreviation FROM teams", engine)
    merged = df.merge(teams_df, left_on="team_name", right_on="full_name", how="left")

    manual = {
        "Washington Commanders": "WSH",
        "San Francisco 49ers": "SF",
        # Add more if needed
    }

    merged["abbreviation"] = merged["abbreviation"].fillna(merged["team_name"].map(manual))
    merged = merged.merge(teams_df[['team_id', 'abbreviation']], on='abbreviation', how='left')

    final = merged[['season', 'team_id', f"{unit}_score", 'source']].dropna(subset=['team_id']).drop_duplicates()

    final.to_sql(f"{unit}_rankings", engine, if_exists="replace", index=False)
    print(f"✅ Ingested {unit} rankings for season {season}")
